# Import liberies

In [ ]:
import warnings
import sklearn
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf 
import keras 
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, GlobalAvgPool2D, Conv2D
from tensorflow.keras import optimizers, callbacks
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
# from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.metrics import confusion_matrix, accuracy_score
from PIL import Image, ImageOps, ImageEnhance

import os
import re
import pandas as pd
import numpy as np

# Data processing

## Prepare

Convert Images to train test dataset

In [ ]:

# def convert_images_to_dataset():
#     ls = []
#     image_data=[]
#     for root, _, files in os.walk("/kaggle/input/rice-leaf-diseases"):
#         if len(root) > len("/kaggle/input/rice-leaf-diseases/rice_leaf_diseases"):
#             label = root[len("/kaggle/input/rice-leaf-diseases/rice_leaf_diseases")+1:]
#         for filename in files:
#             image_path = os.path.join(root, filename)
#             image_data.append(image_path)
#             if label == "Bacterial leaf blight":
#                 ls.append(0)
#             elif label == "Brown spot":
#                 ls.append(1)
#             elif label == "Leaf smut":
#                 ls.append(2)
#     return image_data, ls
# image_data, ls = convert_images_to_dataset()
# df = pd.DataFrame()
# df["image_path"] = image_data
# df["label"] = ls
# df = df.sample(frac=1, random_state=42)  # Set a random_state for reproducibility
# df.reset_index(drop=True, inplace=True)
# df.head()

In [ ]:
def info_folder(folder):
    for i in os.listdir(folder):
        print(i,len(os.listdir(folder+"/"+i)))

In [ ]:
info_folder("/kaggle/input/rice-diseases-dataset/dataset2/train")

## Transform

In [ ]:
# ohencoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
# ohlabel = pd.DataFrame(ohencoder.fit_transform(df[['label']]),dtype = 'float64',columns = ['label0','label1','label2'])
# label_X = df.copy()
# df = pd.concat([df,ohlabel],axis = 1)
# new_df = df.drop(['label'],axis = 1)

## Split

With dataframe

In [ ]:
def resized_image(data):
    flat = []
    for i in data:
        image = Image.open(i)
        width, height = image.size
        new_width = 224
        new_height = 224
        resized_image = image.resize((new_width, new_height))
        img_array = np.array(resized_image) / 255.0
        flat.append(img_array)
    processed_images = np.array(flat).reshape(-1, 224, 224, 3)
    return processed_images

In [ ]:
# train,test = train_test_split(new_df,test_size=0.2,random_state=32,shuffle = True)
# X_train = train['image_path'].values
# y_train = train.drop(['image_path'],axis=1).values
# X_test = test['image_path'].values
# y_test = test.drop(['image_path'],axis=1).values
# dx_train = resized_image(X_train)
# dx_test = resized_image(X_test)

With directly in folder

In [ ]:
# run this on local pc since kaggle doesn't allow to change directory element
# def split_train_test(folder):   
#     try:
#         os.mkdir("train")
#         os.mkdir("test")
#     except:
#         pass
#     for i in os.listdir(folder):
#         os.mkdir("train/"+i)
#         os.mkdir("test/"+i)
#         for j in os.listdir(folder+"/"+i)[:400]:
#             os.rename(folder+"/"+i+"/"+j, "train/"+i+"/"+j)
#         for j in os.listdir(folder+"/"+i)[:100]:
#             os.rename(folder+"/"+i+"/"+j, "test/"+i+"/"+j)

In [ ]:
# split_train_test("/kaggle/input/rice-leaf-images/rice_images")

In [ ]:
def img_data(dir_path,
             target_size,
             batch_size,
             class_lst,
             preprocessing,
            _type):
    if preprocessing:
        if _type == "train":
            gen_object = ImageDataGenerator(rescale = 1./255,
                                            shear_range = 0.2,
                                            zoom_range = 0.2,
                                            horizontal_flip = True,
                                            preprocessing_function=preprocessing)
        else:
            gen_object = ImageDataGenerator(rescale = 1./255,
                                            preprocessing_function=preprocessing)
    else:
        gen_object = ImageDataGenerator()
        
    
        
    return gen_object.flow_from_directory(dir_path,
                                      target_size=target_size,
                                      batch_size=batch_size, 
                                      classes=class_lst,
                                      shuffle=True)



In [ ]:
class_lst = os.listdir("/kaggle/input/rice-diseases-dataset/dataset2/test")

In [ ]:
class_lst

In [ ]:
train_data_gen =  img_data("/kaggle/input/rice-diseases-dataset/dataset2/train",
                          (224,224),
                           20,
                           class_lst,
                          preprocess_input,
                          "train")
test_data_gen =  img_data("/kaggle/input/rice-diseases-dataset/dataset2/test",
                          (224,224),
                           20,
                           class_lst,
                          preprocess_input,
                         "test")

# Model training

In [ ]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50V2(
                   include_top=False,
                   input_shape=(224,224,3),
                   pooling=None,classes=20,
                   weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
# resnet_model.add(GlobalAvgPool2D())
resnet_model.add(Flatten())
resnet_model.add(Dense(1024, activation='relu'))
resnet_model.add(Dense(len(class_lst), activation='softmax'))

In [ ]:
resnet_model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
resnet_model.summary()

In [ ]:
elst = callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')
save_ck =callbacks.ModelCheckpoint('.mdl_wt.hdf5',save_best_only=True, monitor='val_loss',mode='min')

In [ ]:
history = resnet_model.fit(train_data_gen,batch_size=16,epochs=50,validation_data=test_data_gen,callbacks=[elst])


# Model evalution

In [ ]:
# plot the loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

# communcating result

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
resnet_model.save('model_resnet50V2')

In [ ]:
# import pickle

# with open("modelresnet50.pkl", "wb") as model_file:
#     pickle.dump(resnet_model, model_file)

In [ ]:
# with open("model_resnet50", "rb") as model_file:
#     model = pickle.load(model_file)

In [ ]:
model=load_model('model_resnet50V2')

In [ ]:
img=image.load_img('/kaggle/input/rice-diseases-dataset/dataset2/train/blight/IMG_0937.jpg',target_size=(224,224))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)


In [ ]:
img_data=preprocess_input(x)
a=np.argmax(model.predict(img_data), axis=1)

In [ ]:
print(a,model.predict(img_data)[0])

In [ ]:
pdict = {}
for i, k in enumerate(class_lst):
    pdict[i] = k

In [ ]:
pdict

In [ ]:
prediction = model.predict(img_data)
pred = np.argmax(prediction[0])
label=[-1]

In [ ]:
plt.imshow(img)
plt.title("prediction : {0}  {1:.2f}% confidence".format(pdict[pred],prediction[0,pred]*100))
print("prediction : {0}  {1:.2f}% confidence".format(pdict[pred],prediction[0,pred]*100))

In [ ]:
def model_prediction(imagepath,model, pdict):
    pdict = pdict
    img=image.load_img(imagepath,target_size=(224,224))
    image_array=image.img_to_array(img)
    x=np.expand_dims(image_array,axis=0)    
    pred_x = preprocess_input(x)
    prediction = model.predict(pred_x)
    pred = np.argmax(prediction[0])
    result = "prediction : {0}  {1:.2f}% confidence".format(pdict[pred],prediction[0,pred]*100)
    return result

In [ ]:
image_path = "/kaggle/input/rice-leafs-disease-dataset/RiceLeafsDisease/train/bacterial_leaf_blight/bacterial_leaf_blight (107).JPG"
result = model_prediction(image_path,model, pdict)
print(result)

### Creating requirements.txt

In [ ]:
print("scikit-learn==", sklearn.__version__)
print("TensorFlow==", tf.__version__)
print("Keras version==", keras.__version__)
print("Pillow==", Image.__version__)
print("pandas==", pd.__version__)
print("NumPy==", np.__version__)